In [55]:
#Imports
import keras
import os
import numpy as np

In [56]:
#Declare possible labels
colors = []
labels = [
        "red",
        "green",
        "blue",
        "orange",
        "yellow",
        "pink",
        "purple",
        "brown",
        "grey"
    ]
labelsList = []

In [57]:
#Load data and labels from clean_data.txt
filepath = "clean_data.txt"
if not os.path.isfile(filepath):
    print("File path {} does not exist. Exiting...".format(filepath))
    sys.exit()
with open(filepath) as fp:
    i = 0
    r = 0
    g = 0
    b = 0
    for line in fp:
        word = line.split(",")
        col = [float(word[0]) / 255, float(word[1]) / 255, float(word[2]) / 255]
        colors.append(col)
        labelsList.append(labels.index(word[3].rstrip("\n")))
            

In [58]:
#Change input data to numpy array and labels to a one-hot vector encoding
xs = np.array(colors)
print(xs)
labelsList = np.array(labelsList)
ys = keras.backend.one_hot(labelsList, 9)
print(ys)


[[0.31764706 0.71764706 0.60784314]
 [0.97647059 0.08627451 0.27843137]
 [0.99607843 0.76862745 0.12941176]
 ...
 [0.98431373 0.25882353 0.61960784]
 [0.56470588 0.53333333 0.41568627]
 [0.97647059 0.88627451 0.12941176]]
tf.Tensor(
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(5643, 9), dtype=float32)


In [59]:
#Create architecture for model
model = keras.Sequential()
    
model.add(keras.layers.Dense(units=16, activation='sigmoid', input_shape=[3]))

model.add(keras.layers.Dense(units=9, activation='softmax'))

opt = keras.optimizers.SGD(learning_rate=0.2)

model.compile(optimizer=opt, loss='categorical_crossentropy')


In [ ]:
#Train model
model.fit(x=xs,y=ys, epochs=500)

Epoch 1/500
177/177 [==============================] - 0s 484us/step - loss: 1.8985
Epoch 2/500
177/177 [==============================] - 0s 473us/step - loss: 1.6801
Epoch 3/500
177/177 [==============================] - 0s 470us/step - loss: 1.4375
Epoch 4/500
177/177 [==============================] - 0s 471us/step - loss: 1.2616
Epoch 5/500
177/177 [==============================] - 0s 468us/step - loss: 1.1516
Epoch 6/500
177/177 [==============================] - 0s 467us/step - loss: 1.0825
Epoch 7/500
177/177 [==============================] - 0s 468us/step - loss: 1.0356
Epoch 8/500
177/177 [==============================] - 0s 469us/step - loss: 1.0008
Epoch 9/500
177/177 [==============================] - 0s 466us/step - loss: 0.9748
Epoch 10/500
177/177 [==============================] - 0s 469us/step - loss: 0.9549
Epoch 11/500
177/177 [==============================] - 0s 469us/step - loss: 0.9374
Epoch 12/500
177/177 [==============================] - 0s 473us/step - lo

177/177 [==============================] - 0s 466us/step - loss: 0.7631
Epoch 98/500
177/177 [==============================] - 0s 467us/step - loss: 0.7622
Epoch 99/500
177/177 [==============================] - 0s 472us/step - loss: 0.7607
Epoch 100/500
177/177 [==============================] - 0s 479us/step - loss: 0.7608
Epoch 101/500
177/177 [==============================] - 0s 471us/step - loss: 0.7610
Epoch 102/500
177/177 [==============================] - 0s 492us/step - loss: 0.7578
Epoch 103/500
177/177 [==============================] - 0s 499us/step - loss: 0.7580
Epoch 104/500
177/177 [==============================] - 0s 465us/step - loss: 0.7571
Epoch 105/500
177/177 [==============================] - 0s 464us/step - loss: 0.7558
Epoch 106/500
177/177 [==============================] - 0s 464us/step - loss: 0.7558
Epoch 107/500
177/177 [==============================] - 0s 467us/step - loss: 0.7552
Epoch 108/500
177/177 [==============================] - 0s 469us/step

177/177 [==============================] - 0s 488us/step - loss: 0.7059
Epoch 193/500
177/177 [==============================] - 0s 487us/step - loss: 0.7060
Epoch 194/500
177/177 [==============================] - 0s 486us/step - loss: 0.7057
Epoch 195/500
177/177 [==============================] - 0s 485us/step - loss: 0.7039
Epoch 196/500
177/177 [==============================] - 0s 491us/step - loss: 0.7039
Epoch 197/500
177/177 [==============================] - 0s 480us/step - loss: 0.7042
Epoch 198/500
177/177 [==============================] - 0s 482us/step - loss: 0.7029
Epoch 199/500
177/177 [==============================] - 0s 488us/step - loss: 0.7025
Epoch 200/500
177/177 [==============================] - 0s 492us/step - loss: 0.7022
Epoch 201/500
177/177 [==============================] - 0s 486us/step - loss: 0.7008
Epoch 202/500
177/177 [==============================] - 0s 489us/step - loss: 0.7011
Epoch 203/500
177/177 [==============================] - 0s 490us/st

In [ ]:
#Specify what rgb value you would like to test
predict = [[(28/255), (28/255), (201/255)]]

In [ ]:
#Make model prediction
prediction = model.predict(predict)
prediction = keras.backend.argmax(prediction, axis=-1)
prediction = labels[int(prediction)]
print(prediction)